# Wood density grid search by random forest models in GEE

In [9]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [10]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()
os.chdir('/Users/LeonidMoore/Desktop/ETH_Study/WoodDensityMapingProject')

In [11]:
# initialize the earth engine API
ee.Initialize()

In [12]:
# Load the composite for root shoot ratio analysis
compositeRaw = ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
# get the projection
stdProj = compositeRaw.projection()
forestAgeData = ee.Image("projects/crowtherlab/johan/Besnard_ForestAge")
print(colored('Band in Forest age data:\n', 'blue', attrs=['bold']),forestAgeData.bandNames().getInfo())
# we choose the band "forest_age_TC030" for following modeling
forestAge = forestAgeData.select(['forest_age_TC000']).reproject(crs=stdProj).rename('ForestAge')
# load the additional layers and uniform the projection
soilmoisture = ee.Image('users/haozhima95/wld_soil_moisture').reproject(crs=stdProj).rename('SoilMoisture')
compositeImg = compositeRaw.addBands(soilmoisture).addBands(forestAge)
# check the band names in the composite
compositeBandNames = compositeImg.bandNames()
print(colored('Band in composite:\n', 'blue', attrs=['bold']),compositeBandNames.getInfo())

Band in Forest age data:
 ['forest_age_TC000', 'forest_age_TC010', 'forest_age_TC020', 'forest_age_TC030', 'forest_age_TCloss_intensity', 'forest_age_LastTimeTCloss_std']
Band in composite:
 ['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month', 'CHELSA_Mean_Diurnal_Range', 'CHELSA_Mean_Temperature_of_Coldest_Quarter', 'CHELSA_Mean_Temperature_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Warmest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'CHELSA_Min_Temperature_of_Coldest_Month', 'CHELSA_Precipitation_Seasonality', 'CHELSA_Precipitation_of_Coldest_Quarter', 'CHELSA_Precipitation_of_Driest_Month', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Precipitation_of_Warmest_Quarter', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'CHELSA_Temperature_Annual_Range', 'CHELSA_Temperature_Seasonality', 'Depth_to_Water_Table', 'EarthEnvCloudCover_MOD

In [13]:
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

In [14]:
# define the list of predictors
predictorsSelected = ["Aridity_Index",
                      "CHELSA_Annual_Mean_Temperature",
                      "CHELSA_Annual_Precipitation",
                      "CHELSA_Isothermality",
                      "CHELSA_Max_Temperature_of_Warmest_Month",
                      "CHELSA_Mean_Diurnal_Range",
                      "CHELSA_Mean_Temperature_of_Coldest_Quarter",
                      "CHELSA_Mean_Temperature_of_Driest_Quarter",
                      "CHELSA_Mean_Temperature_of_Warmest_Quarter",
                      "CHELSA_Mean_Temperature_of_Wettest_Quarter",
                      "CHELSA_Min_Temperature_of_Coldest_Month",
                      "CHELSA_Precipitation_Seasonality",
                      "CHELSA_Precipitation_of_Coldest_Quarter",
                      "CHELSA_Precipitation_of_Driest_Month",
                      "CHELSA_Precipitation_of_Driest_Quarter",
                      "CHELSA_Precipitation_of_Warmest_Quarter",
                      "CHELSA_Precipitation_of_Wettest_Month",
                      "CHELSA_Precipitation_of_Wettest_Quarter",
                      "CHELSA_Temperature_Annual_Range",
                      "CHELSA_Temperature_Seasonality",
                      "Depth_to_Water_Table",
                      "EarthEnvCloudCover_MODCF_interannualSD",
                      "EarthEnvCloudCover_MODCF_intraannualSD",
                      "EarthEnvCloudCover_MODCF_meanannual",
                      "EarthEnvTopoMed_Eastness",
                      "EarthEnvTopoMed_Elevation",
                      "EarthEnvTopoMed_Northness",
                      "EarthEnvTopoMed_ProfileCurvature",
                      "EarthEnvTopoMed_Roughness",
                      "EarthEnvTopoMed_Slope",
                      "EarthEnvTopoMed_TopoPositionIndex",
                      "SG_Absolute_depth_to_bedrock",
                      "WorldClim2_SolarRadiation_AnnualMean",
                      "WorldClim2_WindSpeed_AnnualMean",
                      "WorldClim2_H2OVaporPressure_AnnualMean",
                      "NDVI",
                      "EVI",
                      "Lai",
                      "Fpar",
                      "Npp",
                      "Tree_Density",
                      "PET",
                      "SG_Clay_Content_0_100cm",
                      "SG_Coarse_fragments_0_100cm",
                      "SG_Sand_Content_0_100cm",
                      "SG_Silt_Content_0_100cm",
                      "SG_Soil_pH_H2O_0_100cm",
                      "LandCoverClass_Cultivated_and_Managed_Vegetation",
                      "LandCoverClass_Urban_Builtup",
                      "Human_Disturbance",
                      "PresentTreeCover",
                      "Nitrogen",
                      "CanopyHeight",
                      "cropland",
                      "grazing",
                      "pasture",
                      "rangeland",
                      "Fire_Frequency",
                      "cnRatio",
                      "Cation",
                      "SoilMoisture",
                      "ForestAge"]
# show the predictors
print(colored('The predictors are:', 'blue', attrs=['bold']),predictorsSelected)

The predictors are: ['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month', 'CHELSA_Mean_Diurnal_Range', 'CHELSA_Mean_Temperature_of_Coldest_Quarter', 'CHELSA_Mean_Temperature_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Warmest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'CHELSA_Min_Temperature_of_Coldest_Month', 'CHELSA_Precipitation_Seasonality', 'CHELSA_Precipitation_of_Coldest_Quarter', 'CHELSA_Precipitation_of_Driest_Month', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Precipitation_of_Warmest_Quarter', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'CHELSA_Temperature_Annual_Range', 'CHELSA_Temperature_Seasonality', 'Depth_to_Water_Table', 'EarthEnvCloudCover_MODCF_interannualSD', 'EarthEnvCloudCover_MODCF_intraannualSD', 'EarthEnvCloudCover_MODCF_meanannual', 'EarthEnvTopoMed_Eastness', 'EarthEnvTopoMed_Elevation', 'EarthEnvTopoM

In [15]:
# Define a function to take a feature with a classifier of interest
def computeCVAccuracy(featureWithClassifier,
                      propertyOfInterest,
                      modelType,
                      kFoldAssignmentFC,
                      cvFoldString,
                      classProperty,
                      extractedVariableTable):
    # Pull the classifier from the feature
    cOI = ee.Classifier(featureWithClassifier)
    # Create a function to map through the fold assignments and compute the overall accuracy
    # for all validation folds
    def computeAccuracyForFold(foldFeature):
        # Organize the training and validation data
        foldNumber = ee.Number(ee.Feature(foldFeature).get('Fold'))
        # print(foldNumber.getInfo())
        trainingData = extractedVariableTable.filterMetadata(cvFoldString,'not_equals',foldNumber)
        # print(trainingData.first().getInfo())
        validationData = extractedVariableTable.filterMetadata(cvFoldString,'equals',foldNumber)
        # Train the classifier and classify the validation dataset
        trainedClassifier = cOI.train(trainingData,classProperty,propertyOfInterest)
        outputtedPropName = 'Predicted'
        classifiedValidationData = validationData.classify(trainedClassifier,outputtedPropName)
        # Create a central if/then statement that determines the type of accuracy values that are returned
        copyProps = ['x','y','WdDnsty','CV_Fold']
        return classifiedValidationData.copyProperties(source = trainingData,properties = copyProps) #.set('Fold',ee.Number(foldNumber))

    # Compute the accuracy values of the classifier across all folds
    accuracyFC = kFoldAssignmentFC.map(computeAccuracyForFold).flatten()
    
    return accuracyFC

In [ ]:
cvFoldString = 'CV_Fold'
classProperty = 'WdDnsty'
# define a loop through the seed list
seedList = np.arange(0, 200, 1).tolist()
print(colored('The models are:', 'blue', attrs=['bold']),seedList)
print(colored('Model is running:\nWith paramter sets:', 'blue', attrs=['bold']))
# for seed in seedList: range(0,len(seedList))
for seed in seedList:
    # load the points data in shape file format
    woodDensityPoints = ee.FeatureCollection('users/leonidmoore/WoodDensityProject/TrainTables/Wood_Density_BufferZone_Subsampled_Train_table_Seed_'+str(seed))
    # check the information of the FeatureCollection
    # print(woodDensityTable.first().getInfo())
    # extract the environment covariates
    extractedVariableTable = compositeImg.select(predictorsSelected).reduceRegions(collection = woodDensityPoints,
                                                                                     reducer = ee.Reducer.first())
    # show the str of the extracted data
    # print(extractedVariableTable.first().getInfo())
    # exclude the rows with null valus
    trainTable = extractedVariableTable.filter(ee.Filter.notNull(predictorsSelected))
    # print(trainTable.size().getInfo())
    parameterTable = pd.read_csv('Data/GridSearchResultsGEE/Wood_Density_Buffer_Based_Subsample_Grid_Search_Seed_'+str(seed)+'.csv', float_precision='round_trip')
    # not recomend to run the code below
    # print(parameterTable.head())
    # extract the paramters
    variablesPerSplitVal = int(parameterTable['variablesPerSplit'].iat[0]) # mtry
    minLeafPopulationVal = int(parameterTable['minLeafPopulation'].iat[0]) # minrow
    maxNodesVal = int(parameterTable['maxNodes'].iat[0]) # mac depth
    print('seed',seed,variablesPerSplitVal,minLeafPopulationVal,maxNodesVal)
    # define the random forest classifier
    rfClassifier = ee.Classifier.smileRandomForest(numberOfTrees = 250,
                                                   variablesPerSplit = variablesPerSplitVal, # mtry
                                                   minLeafPopulation = minLeafPopulationVal, # minrow
                                                   maxNodes = maxNodesVal, # max depth
                                                   bagFraction = 0.632,
                                                   seed = seed).setOutputMode('REGRESSION')
    kList = list(range(0,10))
    kFoldAssignmentFC = ee.FeatureCollection(ee.List(kList).map(lambda n: ee.Feature(ee.Geometry.Point([0,0])).set('Fold',n)))
    
    predictedAndObs = computeCVAccuracy(rfClassifier,predictorsSelected,modelType='REGRESSION',kFoldAssignmentFC= kFoldAssignmentFC,cvFoldString = cvFoldString,classProperty=classProperty,extractedVariableTable = trainTable)
    # select the target property names
    predObs = predictedAndObs.select(['x','y','WdDnsty','Predicted','CV_Fold'])
    # transfer the feature collection into list for further export
    to_export = predObs.toList(5000).getInfo()
    result = []
    for item in to_export:
        values = item['properties']
        row = [str(values[key]) for key in ['x','y','WdDnsty','Predicted','CV_Fold']]
        row = ",".join(row)
        result.append(row)
    # for loop to write each feature as a row into local folder
    df = pd.DataFrame([item.split(",") for item in result], columns = ['x','y','WdDnsty','Predicted','CV_Fold'])
    with open('Data/PredictAndObs/Exported_Table_of_Prediction_and_Observation_with_10_fold_for_'+str(seed)+'.csv', 'a') as f:
        df.to_csv(f, mode='a', header=f.tell()==0)
    # show the progress
    print(colored('Prediction of model'+str(seed)+' finished!', 'blue', attrs=['bold']))


The models are: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
Model is running:
With paramter sets:
seed 0 15 2 90
Prediction of model0 finished!
seed 1 18

Prediction of model116 finished!
seed 117 15 2 100
Prediction of model117 finished!
seed 118 12 4 100
Prediction of model118 finished!
seed 119 18 10 90
Prediction of model119 finished!
seed 120 15 12 100
Prediction of model120 finished!
seed 121 21 4 100
Prediction of model121 finished!
seed 122 12 2 100
Prediction of model122 finished!
seed 123 18 10 100
Prediction of model123 finished!
seed 124 21 4 100
Prediction of model124 finished!
seed 125 21 6 100
Prediction of model125 finished!
seed 126 21 2 100
Prediction of model126 finished!
seed 127 21 8 90
Prediction of model127 finished!
seed 128 18 8 100
Prediction of model128 finished!
seed 129 21 2 100
Prediction of model129 finished!
seed 130 21 14 100
Prediction of model130 finished!
seed 131 21 12 90
Prediction of model131 finished!
seed 132 18 12 100
Prediction of model132 finished!
seed 133 15 2 100
Prediction of model133 finished!
seed 134 18 6 90
Prediction of model134 finished!
seed 135 21 12 90
Prediction of model135 finish